In [ ]:
import os

from astropy.time import Time
import lightkurve as lk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
'''Get CSV to pandas df'''

def get_csv_file_to_df(directory):
    '''This function gets a csv file and makes a pandas df out of it
    OUTPUT: df of the csv file'''

    if os.path.exists(directory):
        files = os.listdir(directory)
        if len(files) > 0:
            print(f"there are {len(files)} files: {files}, taking '{files[0]}'")
            csv_file_path = os.path.join(directory, files[0]) #take 1st file
            df = pd.read_csv(csv_file_path)
    return df

directory = r'C:\Planetary_Solver\input'
df = get_csv_file_to_df(directory)


In [ ]:
cols_to_drop= ['intentType', 'proposal_type', 'jpegURL', 'dataURL', 'obs_title', 'srcDen', \
               'target_classification', 'dataRights', 'mtFlag']
df_dropped  = df.drop(columns = cols_to_drop)

# ['intentType', 'obs_collection', 'provenance_name', 'instrument_name', 'project', 'filters',
# 'wavelength_region', 'target_name', 'target_classification', 'obs_id', 's_ra', 's_dec',
# 'dataproduct_type', 'proposal_pi', 'calib_level', 't_min', 't_max', 't_exptime', 'em_min',
# 'em_max', 'obs_title', 't_obs_release', 'proposal_id', 'proposal_type', 'sequence_number', 's_region',
#  'jpegURL', 'dataURL', 'dataRights', 'mtFlag', 'srcDen', 'obsid', 'distance']


In [ ]:

pixelfile = lk.search_targetpixelfile("Trappist-1")[1].download()
pixelfile.plot(frame=11)
plt.show()

lc = pixelfile.to_lightcurve(aperture_mask='all')#(method="pld").remove_outliers().flatten()
lc.plot()
plt.show()

# period = lc.to_periodogram("bls").period_at_max_power
# lc.fold(period).truncate(-.5, -.25).scatter()

In [ ]:
# KIC 6922244
pixelFile= lk.search_targetpixelfile('KIC 6922244', quarter=4).download()
lc       = pixelFile.to_lightcurve(aperture_mask= pixelFile.pipeline_mask).remove_nans()
flat_lc  = lc.flatten(window_length=401)

periodogram= flat_lc.to_periodogram(method = 'bls', period= np.arange(0.5, 2, 0.001))
period     = periodogram.period_at_max_power

folded_lc  = flat_lc.fold(period=period)

binned_lc  = folded_lc.bin(binsize = 25)
binned_lc.plot()
plt.show()


In [ ]:
# Pi Mensae
search_result= lk.search_targetpixelfile('Pi Mensae', mission = 'TESS', sector = 1)
tpf          = search_result.download(quality_bitmask = 'default')
lc           = tpf.to_lightcurve()

aperture_mask= tpf.create_threshold_mask(threshold = 10)
lc           = tpf.to_lightcurve(aperture_mask = aperture_mask)

flat_lc = lc.flatten(window_length = 1000)
flat_lc.errorbar()
plt.show()

start_time= Time(1346, format='jd')
end_time  = Time(1350, format='jd')
mask      = (flat_lc.time < start_time) | (flat_lc.time > end_time)
masked_lc = flat_lc[mask]

clipped_lc= masked_lc.remove_outliers(sigma=6)

periodogram= clipped_lc.to_periodogram(method = 'bls', period= np.arange(0.5, 10, 0.001))
period     = periodogram.period_at_max_power

shift_graph_by_days = 1325
folder_lc  = clipped_lc.fold(period = period, t0 = shift_graph_by_days)

binned_lc = folded_lc.bin(binsize = 15)
binned_lc.scatter(s=0.1)
plt.show()
